In [6]:
import pandas as pd
import datetime as dt

In [7]:
df = pd.read_csv("data/dow.csv", index_col=[0,1])

In [9]:
def log(f):
    def wrapper(df, *args, **kwargs):
        start = dt.datetime.now()
        result = f(df, *args, **kwargs)
        stop = dt.datetime.now()
        print(f'{f.__name__}:\n  runtime={stop - start}, end shape={result.shape}')
        return result
    return wrapper

@log
def startPipeline(df):
    return df.copy()

@log
def clean(df):
    return df

@log 
def removeOutliers(df):
    return df

df_processed = (df
 .pipe(startPipeline)
 .pipe(clean)
 .pipe(removeOutliers))
df_processed

startPipeline:
  runtime=0:00:00, end shape=(62, 8)
clean:
  runtime=0:00:00, end shape=(62, 8)
removeOutliers:
  runtime=0:00:00, end shape=(62, 8)


Open        High         Low       Close      Volume  \
Symbol Date                                                                     
AA     1998-01-20   28.716094   29.547440   28.681454   29.530121   2176900.0   
       2021-01-14   25.000000   25.379999   24.850100   25.090000   3948281.0   
AIG    1998-01-20  482.618128  486.123816  476.775276  485.831665    207900.0   
       2021-01-14   41.000000   41.448101   40.680000   41.299999   3873888.0   
AXP    1998-01-20   17.604445   17.765462   17.537355   17.698372   4635200.0   
...                       ...         ...         ...         ...         ...   
VZ     2021-01-14   56.889999   57.424999   56.660000   57.040001  17987541.0   
WMT    1998-01-20   13.316331   14.115731   13.316331   14.052621   7682800.0   
       2021-01-14  147.820007  148.000000  146.035004  146.970001   6749975.0   
XOM    1998-01-20   15.468195   15.611567   15.372614   15.515985   6348000.0   
       2021-01-14   48.500000   51.080002   48.500000   50.310001  34741787.0   

                   Dividends  Stock Splits  Adj Close  
Symbol Date                                            
AA     1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  
AIG    1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  
AXP    1998-01-20        0.0           0.0        NaN  
...                      ...           ...        ...  
VZ     2021-01-14        0.0           0.0        NaN  
WMT    1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  
XOM    1998-01-20        0.0           0.0        NaN  
       2021-01-14        0.0           0.0        NaN  

[62 rows x 8 columns]